In [1]:
import numpy as np
from functools import partial
import ipywidgets as widgets
from IPython.display import display

In [2]:

from pydrake.geometry.optimization import IrisOptions, HPolyhedron, Hyperellipsoid, IrisInConfigurationSpace
from pydrake.solvers import MosekSolver, CommonSolverOption, SolverOptions
from pydrake.all import (PiecewisePolynomial, 
                         InverseKinematics, 
                         Sphere, 
                         Rgba, 
                         RigidTransform, 
                         RotationMatrix, 
                         Solve,
                         MathematicalProgram,
                         SceneGraphCollisionChecker,
                         RollPitchYaw,
                         Cylinder)
import time
import pydrake
import pickle
from visibility_utils import generate_distinct_colors 
from visualization_utils import plot_points

/home/peter/.local/lib/python3.8/site-packages/scipy/__init__.py:143: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.17.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
from environments import get_environment_builder

plant_builder = get_environment_builder('7DOFBINS')
plant, scene_graph, diagram, diagram_context, plant_context, meshcat = plant_builder(usemeshcat=True)

scene_graph_context = scene_graph.GetMyMutableContextFromRoot(
    diagram_context)
robot_instances = [plant.GetModelInstanceByName("iiwa"), plant.GetModelInstanceByName("wsg")]
checker = SceneGraphCollisionChecker(model = diagram,#.Clone(), 
                    robot_model_instances = robot_instances,
                    distance_function_weights =  [1] * plant.num_positions(),
                    #configuration_distance_function = _configuration_distance,
                    edge_step_size = 0.125)

INFO:drake:Meshcat listening for connections at http://localhost:7001
INFO:drake:Allocating contexts to support 20 parallel queries given omp_num_threads 20 omp_max_threads 20 and omp_thread_limit 2147483647 OpenMP enabled in build? true


In [4]:
def show_pose(qvis, plant, plant_context, diagram, diagram_context, endeff_frame, show_body_frame = None):
    plant.SetPositions(plant_context, qvis)
    diagram.ForcedPublish(diagram_context)
    tf =plant.EvalBodyPoseInWorld(plant_context,  plant.GetBodyByName(endeff_frame))
    transl = tf.translation()+tf.rotation()@np.array([0,0.1,0])
    if show_body_frame is not None:
        show_body_frame(RigidTransform(tf.rotation(), transl))

def show_ik_target(pose, meshcat, name):
    h = 0.2
    if 'targ' in name:
        colors = [Rgba(1,0.5,0, 0.5), Rgba(0.5,1,0, 0.5), Rgba(0.0,0.5,1, 0.5)]
    else:
        colors = [Rgba(1,0,0, 1), Rgba(0.,1,0, 1), Rgba(0.0,0.0,1, 1)]

    rot = pose.rotation()@RotationMatrix.MakeYRotation(np.pi/2)
    pos= pose.translation() +pose.rotation()@np.array([h/2, 0,0])
    meshcat.SetObject(f"/drake/ik_target{name}/triad1",
                                   Cylinder(0.01,0.2),
                                   colors[0])
    meshcat.SetTransform(f"/drake/ik_target{name}/triad1",RigidTransform(rot, pos))
    rot = pose.rotation()@RotationMatrix.MakeXRotation(-np.pi/2)
    pos= pose.translation() +pose.rotation()@np.array([0,h/2,0])

    meshcat.SetObject(f"/drake/ik_target{name}/triad2",
                                   Cylinder(0.01,0.2),
                                   colors[1])
    meshcat.SetTransform(f"/drake/ik_target{name}/triad2",RigidTransform(rot, pos))
    pos= pose.translation().copy()
    rot = pose.rotation()
    pos = pos + rot@np.array([0,0,h/2])
    meshcat.SetObject(f"/drake/ik_target{name}/triad3",
                                   Cylinder(0.01,0.2),
                                   colors[2])
    meshcat.SetTransform(f"/drake/ik_target{name}/triad3",RigidTransform(rot, pos))


def densify_waypoints(waypoints_q, densify = 200):
    dists = []
    dense_waypoints = []
    for idx in range(len(waypoints_q[:-1])):
        a = waypoints_q[idx]
        b = waypoints_q[idx+1]
        t = np.linspace(1,0, 10)
        locs_endeff = []
        dists_endeff = []
        for tval in t:
            a = a*tval + b*(1-tval)
            qa = a#Ratfk.ComputeQValue(ta, np.zeros(7))
            #showres(qa)
            #time.sleep(0.1)            
            plant.SetPositions(plant_context, qa)
            tf_tot= plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName('body'))
            tf = tf_tot.translation() + tf_tot.GetAsMatrix4()[:3,:3][:,1] *0.13
            locs_endeff.append(tf)
        for i in range(len(locs_endeff)-1):
            dists_endeff.append(np.linalg.norm(locs_endeff[i]- locs_endeff[i+1]))
        d = np.sum(dists_endeff)
        #print(d * densify)
        t = np.linspace(1,0,int(densify*d))
        for tval in t:
            dense_waypoints.append(waypoints_q[idx]*tval + waypoints_q[idx+1]*(1-tval))
    return dense_waypoints

def plot_endeff_traj(dense_waypoints, name = '', color= (1,0,0,1)):
    color = Rgba(color[0], color[1], color[2], color[3])
    start_idx = 0
    for i, qa in enumerate(dense_waypoints):
        #qa = Ratfk.ComputeQValue(ta, np.zeros(7))
        #showres(qa)
        #time.sleep(0.1)            
        plant.SetPositions(plant_context, qa)
        tf_tot= plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName('body'))
        tf = tf_tot.translation() + tf_tot.GetAsMatrix4()[:3,:3][:,1] *0.13

        meshcat.SetObject(f"/iris/points/traj/{name}/{i+start_idx}",
                               Sphere(0.01),
                               color)

        meshcat.SetTransform(f"/iris/points/traj/{name}/{i+start_idx}",
                                   RigidTransform(RotationMatrix(),
                                                  tf))

In [5]:
show_body_frame = partial(show_ik_target, 
                          meshcat=meshcat, 
                          name='endeff_acutal', 
                          )
showres = partial(show_pose, 
                  plant = plant, 
                  plant_context = plant_context, 
                  diagram = diagram, 
                  diagram_context = diagram_context,
                  endeff_frame = 'body',
                  show_body_frame=show_body_frame)

In [6]:
q = np.zeros(plant.num_positions())

sliders = []
for i in range(plant.num_positions()):
    q_low = plant.GetPositionLowerLimits()[i]
    q_high = plant.GetPositionUpperLimits()[i]
    sliders.append(widgets.FloatSlider(min=q_low, max=q_high, value=0, description=f"q{i}"))

def handle_slider_change(change, idx):
    q[idx] = change['new']
    showres(q)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)

FloatSlider(value=0.0, description='q0', max=2.96706, min=-2.96706)

FloatSlider(value=0.0, description='q1', max=2.0944, min=-2.0944)

FloatSlider(value=0.0, description='q2', max=2.96706, min=-2.96706)

FloatSlider(value=0.0, description='q3', max=2.0944, min=-2.0944)

FloatSlider(value=0.0, description='q4', max=2.96706, min=-2.96706)

FloatSlider(value=0.0, description='q5', max=2.0944, min=-2.0944)

FloatSlider(value=0.0, description='q6', max=3.05433, min=-3.05433)

In [7]:
from task_space_seeding_utils import solve_ik_problem

In [8]:
#ik sliders 
show_ik_targets = partial(show_ik_target, 
                          meshcat=meshcat, 
                          name='endeff_targ', 
                          )

ik_solver = partial(solve_ik_problem, 
                    plant_ik = plant,
                    plant_context_ik = plant_context,
                    frames = [plant.GetFrameByName('body')],
                    offsets = [np.array([0,0.1,0])],
                    collision_free=False,
                    track_orientation=False)
sliders_ik = []
pos_min = np.array([-0.5,-0.5, 0.01, -np.pi,-np.pi, -np.pi])
pos_max = np.array([1.0,0.5, 1, np.pi,np.pi, np.pi])
names = ['x', 'y', 'z','rx', 'ry', 'rz']
q0 = np.zeros(7)#np.array([ 0.23294, -0.2944 , -0.36706, -1.5944 , -0.16706,  0.     ,
       # 1.64567])
_ik_positions = np.zeros(6)

for i in range(6):
    sliders_ik.append(widgets.FloatSlider(min=pos_min[i], 
                                       max=pos_max[i], 
                                       value=0.5*(pos_max[i]-pos_min[i])+pos_min[i], 
                                       description=names[i]))

def handle_slider_change_ik(change, idx):
    _ik_positions[idx] = change['new']
    rot = RotationMatrix.MakeXRotation(_ik_positions[3])@RotationMatrix.MakeYRotation(_ik_positions[4])@RotationMatrix.MakeZRotation(_ik_positions[5])

    RollPitchYaw(_ik_positions[3],
                       _ik_positions[4],
                       _ik_positions[5]).ToRotationMatrix()
    tf = RigidTransform(rot, _ik_positions[:3])
    show_ik_targets(tf)
    res = ik_solver([tf], q0)
    if res is not None:
        showres(res)

idx = 0
for slider in sliders_ik:
    slider.observe(partial(handle_slider_change_ik, idx = idx), names='value')
    idx+=1

for slider in sliders_ik:
    display(slider)

FloatSlider(value=0.25, description='x', max=1.0, min=-0.5)

FloatSlider(value=0.0, description='y', max=0.5, min=-0.5)

FloatSlider(value=0.505, description='z', max=1.0, min=0.01)

FloatSlider(value=0.0, description='rx', max=3.141592653589793, min=-3.141592653589793)

FloatSlider(value=0.0, description='ry', max=3.141592653589793, min=-3.141592653589793)

FloatSlider(value=0.0, description='rz', max=3.141592653589793, min=-3.141592653589793)

# Load interesting seeding data for IRIS

In [37]:
# interesting_indices = [18,100, 166, 191, 224, 227, 238]
interesting_indices = [100, 166, 238]  
with open('seed_data.pkl', 'rb') as f:
    d = pickle.load(f)
    seed_points = [d['seed_points'][i] for i in interesting_indices]
    metrics = [d['ellipsoids'][i] for i in interesting_indices]
    clique_points = [d['cliques'][i] for i in interesting_indices]
    task_space_clique_points = [d['tasksp_cliques'][i] for i in interesting_indices]
    regs_coll = d['regs_crit']

# show interesting cliques

In [19]:

cols = generate_distinct_colors(len(clique_points)+5, rgb = True)[5:]
cols = [list(c)+[1] for c in cols]
for id in range(len(clique_points)):
    plot_points(meshcat, task_space_clique_points[id], f"ct{id}", size = 0.01, color=Rgba(*cols[id]))
for idx in range(len(clique_points)):
        meshcat.SetProperty(f"/drake/ct{idx}", "visible", True)

clique_idx = 0
for clique_idx in range(len(clique_points)):
    for idx in range(len(clique_points)):
        meshcat.SetProperty(f"/drake/ct{idx}", "visible", False)
    meshcat.SetProperty(f"/drake/ct{clique_idx}", "visible", True)
        
    cliques_vis = clique_points[clique_idx]
    start = cliques_vis[0]
    target = cliques_vis[np.random.randint(len(cliques_vis))]
    showres(target)
    for i in range(10):
        wpd = densify_waypoints([start, target], 50)
        #print(f"norm {np.linalg.norm(start-target)}")
        plot_endeff_traj(wpd, 'p', cols[clique_idx])
        for qa in wpd:
            showres(qa)
            #diagram_context.SetTime(cur_time)
            # if col_func_handle(qa):
            #     print('col')
            # cur_time+=frame_time
            time.sleep(0.02)
        meshcat.Delete('/iris/points/traj')
        start = target
        target = cliques_vis[np.random.randint(len(cliques_vis))]

# Set up IRIS

In [43]:
snopt_iris_options = IrisOptions()
snopt_iris_options.require_sample_point_is_contained = True
snopt_iris_options.iteration_limit = 1
snopt_iris_options.configuration_space_margin = 1.e-4
#snopt_iris_options.max_faces_per_collision_pair = 60
snopt_iris_options.termination_threshold = -1
#snopt_iris_options.q_star = np.zeros(3)
snopt_iris_options.num_collision_infeasible_samples = 80
snopt_iris_options.relative_termination_threshold = 0.02
#snopt_iris_options.num_snopt_seed_guesses = 1

In [44]:
regions = []
for i, (s,m) in enumerate(zip(seed_points, metrics)):
    plant.SetPositions(plant_context, s)
    snopt_iris_options.starting_ellipse = m
    try:
        #r = IrisInConfigurationSpace(plant, diagram_context, snopt_iris_options)
        r = IrisInConfigurationSpace(plant, plant_context, snopt_iris_options)
        regions.append(r)
    except Exception as e:
        print(f"failed to generate region {i} because \n{e}")

INFO:drake:IrisInConfigurationSpace iteration 0
INFO:drake:IrisInConfigurationSpace iteration 0
INFO:drake:IrisInConfigurationSpace iteration 0


In [45]:
from pydrake.all import RandomGenerator
gen = RandomGenerator(4)
Ns = 10000
s_reg = []
for r in regions:
    samps = []
    for _ in range(Ns):
        if len(samps):
            samps.append(r.UniformSample(gen, samps[-1]))
        else:
            
            samps.append(r.UniformSample(gen))
    s_reg.append(samps)

frac_vol_in_collision = []
for s in s_reg:
    tot_col = 0
    for pt in s:
        if not checker.CheckConfigCollisionFree(pt):
            tot_col+=1
    frac_vol_in_collision.append(tot_col/Ns)

print(frac_vol_in_collision)

[0.6712, 0.943, 0.4768]


In [46]:
from pydrake.all import VPolytope
verts = [VPolytope(r).vertices().T for r in regions]

In [52]:

for r in regions:
    qvis = r.ChebyshevCenter()
    gen = RandomGenerator(4)
    for _ in range(10):
        qvis = r.UniformSample(gen, qvis)
        #qvis = v[np.random.choice(len(v))]
        showres(qvis)
        time.sleep(0.5)

In [53]:
for v in verts:
    gen = RandomGenerator(4)
    for _ in range(10):
        qvis = v[np.random.choice(len(v))]
        showres(qvis)
        time.sleep(0.5)